<img src='universe.jpg' style="float: right; height: 600px;" alt="Drawing">
The main goal is to recommend the most similar hotel to a selected hotel. There are other ways of recommending hotels (content-based k-means clustering, for one, as introduced in another tutorial, [hotel_clustering_using_nlp](hotel_clustering_using_nlp.ipynb)). To __look up__ the most similar hotel in our __universe of hotels__, imagine your favorite hotel __lives__ on Planet A. Now when you travel to a different planet (sorry, __city__), you want to find a hotel that reminds you of your favorite on Planet A. A k-dimensional tree algorithm can help you locate that hotel in almost no time. I'll implement an __Approximate Nearest Neighbors__ search on a __7 dimenional tree__ that has been tailored to our dataset of 10,000 hotels that have been rated by over 1.04 million users on TripAdvisor.

In this tutorial, I'll implement a fast search algorithm for content-based k-Nearest Neighbor model. KNN models were known for their interpretability in the early days of recommender systems. I'll define a way of measuring similarity, i.e. how similar any of the chosen two hotels are, based on their features. This similarity measure helps guide us in the __universe__ of __similar hotels__. The model compares similarity measures among hotels, to decide how __similar__ a pair of hotels are in the rating structures given by travelers. If we would like to know how __similar__ a pair of items are, kNN models would also be a very intuitive way to tell the __degree of similarity__. 

In statistics, the __curse of dimensionality__ has made many problems a lot harder when the dimensions increase. In a 1D space (a straight line on paper), this search problem is essential a query problem; whatever hotel located next to your favorite is the most similar one. In a 2D plane (laying out a map on table), we can draw circles centered at your favorite hotel to search for the next similar one. In a 3D world like ours, there is yet another dimension added to the previous plane, so search time increases as a result.

Hotels in our model will be measured in 7 dimensions: rooms, service, cleanlines, front desk, business service, value, location; each will be given a 0 to 5 scores, where 1 - 5 are travelers' ratings. By using a k-dimensional tree, the next similar hotel will be generated automatically from the training set that we feed in to the algorithm.

## Content:
<ul>
<li>[Data pre-processing](#Data pre-processing)
<li>[Define distance metric](#Define distance metric)
<li>[Train test split](#Train test split)
<li>[Build a KD tree](#Build a KD tree)
<li>[Search for the nearest neighbor on a KD tree](#Search for the nearest neighbor on a KD tree)
<li>[Testing on sample](#Testing on sample)
</ul>

I used Python 2 to write this tutorial and implemented the following libraries:

In [1]:
import pandas as pd 
import pprint
import numpy as np
import sklearn.preprocessing as pp
from numpy import linalg

<a id='Data pre-processing'></a>

## Data pre-processing

First, we import our example dataset that I scraped and cleaned from TripAdvisor. Each hotel id is unique and represents a hotel in our databse. Ratings of rooms, service, cleanliness, front desk, business service, value, and location are mean ratings taken from travelers.

In [86]:
hotel_ratings = pd.read_csv('../../dataset/nearest_neighbors_hotel.csv')
hotel_ratings.head()

,hotel id,rooms,service,cleanliness,front desk,business service,value,location,hotel name,city,state,zip code,low price,high price
0,72572,3.644444,4.275556,4.368889,0.760000,0.560000,4.080000,3.822222,BEST WESTERN PLUS Pioneer Square Hotel,Seattle,WA 98104-2530,98104-2530,$117,$189
1,72586,3.130769,3.838462,3.761538,0.553846,0.423077,3.792308,3.615385,BEST WESTERN PLUS Executive Inn,Seattle,WA 98109-5016,98109-5016,$115,$229
2,73855,3.337423,3.607362,3.950920,0.699387,0.257669,3.251534,3.736196,Hyatt Regency Phoenix,Phoenix,AZ 85004,85004,$132,$271
3,73943,3.677852,4.453020,4.442953,1.030201,0.637584,4.036913,3.637584,Royal Palms Resort and Spa,Phoenix,AZ 85018,85018,$182,$501
4,73947,3.207792,4.110390,4.019481,0.305195,0.194805,3.909091,3.396104,Sheraton Crescent Hotel,Phoenix,AZ 85021,85021,$90,$172


<a id='Define distance metric'></a>

## Define distance metric

First, we need to define a way of measuring similarity distance. We choose the Euclidean distance:

In [3]:
def euc_distance(point1, point2):
    diff = np.subtract(point1, point2)
    dist = np.linalg.norm(diff)
    return dist

In [87]:
# we pick the first 3000 hotels to illustrate an application of our algorithm.
# randomly shuffle rows
hotel_ratings = hotel_ratings.sample(frac=1)
hotel_ratings.head()

,hotel id,rooms,service,cleanliness,front desk,business service,value,location,hotel name,city,state,zip code,low price,high price
764,195319,2.509317,3.298137,3.273292,0.962733,0.621118,3.267081,3.658385,Umi London,London W2 4PR,England,W2 4PR,$95,$302
1764,649828,3.899281,4.169065,4.496403,0.280576,0.190647,3.798561,3.255396,Hilton Madrid Airport,28042 Madrid,Spain,28042,$130,$320
1559,478955,1.882353,2.170588,1.988235,0.647059,0.282353,2.117647,3.358824,Tidelands Caribbean Hotel and Suites,Ocean City,MD 21842,21842,$79 and up,null
1319,268228,3.236287,3.932489,4.198312,1.088608,0.586498,3.308017,3.308017,Marriott Waterfront Seattle,Seattle,WA 98121,98121,$209,$337
1221,254348,3.467337,4.472362,4.527638,0.879397,0.462312,4.251256,3.412060,LHotel,Quebec H2Y 1N1,Canada,H2Y 1N1,$183,$263


<a id='Train test split'></a>

## Train test split

In [5]:
#design training and test set after random shuffle:
training = hotel_ratings[:2000]
test = hotel_ratings[2000:]
training.reset_index(inplace=True)
test.reset_index(inplace=True)
training = training[['rooms', 'service', 'cleanliness', 'front desk', 'business service', 'value', 'location']]
test = test[['rooms', 'service', 'cleanliness', 'front desk', 'business service', 'value', 'location']]
training.head()

,rooms,service,cleanliness,front desk,business service,value,location
0,3.888128,4.337900,4.445205,1.344749,0.847032,4.152968,3.484018
1,3.235012,3.774580,4.021583,0.561151,0.179856,3.573141,3.690647
2,3.880383,4.325359,4.287081,0.401914,0.229665,3.928230,4.090909
3,3.563253,4.174699,4.117470,0.683735,0.406627,3.756024,3.852410
4,2.833333,3.371212,3.659091,0.575758,0.469697,3.356061,2.931818


<a id='Build a KD tree'></a>

## Build a KD tree

To generate a tree structure, we define a function that takes arrays of (7-dimensional) points as input, and generates a binary search tree (BST) where each split is performed on one of the 7 dimensions, alternatively, until no more split is available.
<img src='KDTree-animation.gif' style="float: center; height: 400px;" alt="Drawing">

(_Source_: [Wikipedia: kd tree](https://en.wikipedia.org/wiki/K-d_tree))

In [6]:
def build_kdtree(points, depth=0):
    n = len(points)

    if n <= 0:
        return None
    # spliting by alternating axis:
    axis = depth % k

    sorted_points = sorted(points, key=lambda point: point[axis])

    return {
        'point': sorted_points[n / 2],
        'left': build_kdtree(sorted_points[:n / 2], depth + 1),
        'right': build_kdtree(sorted_points[n/2 + 1:], depth + 1)
    }

In [7]:
# turning DataFrames into array structure:
training_points = training.as_matrix()
test_points = test.as_matrix()

Now, we are ready to __grow__ our first tree!

In [8]:
k = 7
kdtree = build_kdtree(training_points)

An example of what the tree looks like:

In [9]:
ppr = pprint.PrettyPrinter(indent=4)
ppr.pprint(kdtree['left']['left']['left']['left']['left']['left']['left']['left'])

{   'left': {   'left': {   'left': None,
                            'point': array([1.40880503, 1.76100629, 1.93710692, 0.19496855, 0.08805031,
       2.01886792, 2.16981132]),
                            'right': None},
                'point': array([1.60493827, 1.83950617, 1.98148148, 0.66666667, 0.2345679 ,
       1.95061728, 2.72222222]),
                'right': {   'left': None,
                             'point': array([1.92322097, 2.32209738, 2.4494382 , 0.52996255, 0.17602996,
       2.64419476, 2.71348315]),
                             'right': None}},
    'point': array([1.8404908 , 2.33128834, 2.14110429, 0.25153374, 0.09815951,
       2.39263804, 1.73619632]),
    'right': {   'left': {   'left': None,
                             'point': array([1.85628743, 2.41916168, 2.25748503, 0.26347305, 0.08982036,
       2.50299401, 1.89221557]),
                             'right': None},
                 'point': array([1.84587814, 2.38351254, 2.73835125, 0.        , 0.   

<a id='Search for the nearest neighbor on a KD tree'></a>

## Search for the nearest neighbor on a KD tree

This tree is a BST where each split is first split on dimension 'rooms', then 'service', etc. until starting from 'rooms' all over again. There is no way human brains can visualize this high dimensional space, but the basic principles are just as simple as the 2D version in the above animation.

To avoid going over all branches in search of the _closest_ points, we apply an approximate version of BST search query algorithm. We first define a function that frees us up by _trimming_ half of the tree each time we detect that none of the child nodes are possible of being in the _closer_ class of our _planet_ (in the universe of hotels)...

In [10]:
# a function that decides if points 'p1' or 'p2' is closer to our 'pivot'
def closer_distance(pivot, p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1

    # calculate the euclidean distances
    d1 = euc_distance(pivot, p1)
    d2 = euc_distance(pivot, p2)
    
    # choose the closer point
    if d1 < d2:
        return p1
    else:
        return p2

Next we need to search for the closest neighbors for a given point on our BST tree.

In [29]:
# a function that searches for the closest neighbors based on a grown BST 'root'.
def kdtree_closest_point(root, point, depth=0):
    if root is None:
        return None
    # alternating split by axis
    axis = depth % k
    
    # initiate search on two branches at the same time
    next_branch = None
    opposite_branch = None
    
    # deciding by preliminary check which branch is worth time investigating
    if point[axis] < root['point'][axis]:
        next_branch = root['left']
        opposite_branch = root['right']
    else:
        next_branch = root['right']
        opposite_branch = root['left']
        
    #initiate recursive process of finding a 'closer_distance' 
    best = closer_distance(point,
                           kdtree_closest_point(next_branch,
                                                point,
                                                depth + 1),
                            root['point'])
    
    # update 'best' point each time a closer point is detected
    if euc_distance(point, best) > abs(point[axis] - root['point'][axis]):
        best = closer_distance(point,
                               kdtree_closest_point(opposite_branch,
                                                    point,
                                                    depth + 1),
                               best)
    # return the closest point
    return best

<a id='Testing on sample'></a>

## Testing on sample

Let's roll up our sleeves and start __climbing__ (searching)!

If I want to know which hotel is most similar to __Hotel Erwin, a Joie de Vivre hotel__ in __Los Angeles, LA__ whose hotel id is __2516007__:

In [66]:
# uncomment the following if you are running this code for the first time
#hotel_ratings.set_index('hotel name', inplace=True)
hotel_ratings.loc["Hotel Erwin, a Joie de Vivre hotel"]

,hotel id,rooms,service,cleanliness,front desk,business service,value,location,city,state,zip code,low price,high price
hotel name,,,,,,,,,,,,,
"Hotel Erwin, a Joie de Vivre hotel",2516007,3.348485,4.126263,4.065657,0.025253,0.025253,3.818182,3.646465,Los Angeles,CA 90291,90291,$256,$404
"Hotel Erwin, a Joie de Vivre hotel",81170,3.310160,4.139037,4.064171,0.026738,0.026738,3.823529,3.636364,Los Angeles,CA 90291,90291,$256,$404


In [67]:
#hotel_ratings.reset_index('hotel id', inplace=True)
hotel_ratings[hotel_ratings['hotel id'] == 2516007]
test_hotel = hotel_ratings[hotel_ratings['hotel id'] == 2516007][['rooms','service', 'cleanliness', 'front desk', 'business service', 
                                                                'value', 'location']].as_matrix()
test_hotel

array([[3.34848485, 4.12626263, 4.06565657, 0.02525253, 0.02525253,
        3.81818182, 3.64646465]])

To find the most similar hotel in the _hotel_ratings_ datafram, we run __kdtree_closest_point__ on our entire __kdtree__, with the __test_hotel__ array as input.

In [73]:
rec_hotel = kdtree_closest_point(kdtree, test_hotel[0])
%time rec_hotel

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10 µs


array([3.34848485, 4.12626263, 4.06565657, 0.02525253, 0.02525253,
       3.81818182, 3.64646465])

In [69]:
recommend = np.argwhere(training_points == rec_hotel)
favorite = np.argwhere(test_points == test_hotel)

In [70]:
hotel_ratings.iloc[recommend[0][0]]

hotel id                  2514393
rooms                     3.41983
service                   3.34402
cleanliness               3.79592
front desk                1.04665
business service         0.676385
value                     3.05831
location                  3.61224
city                New York City
state                    NY 10014
zip code                    10014
low price                    $344
high price                   $589
Name: Hotel Gansevoort, dtype: object

In [71]:
print ('Recommended hotel with hotel id {}'.format(hotel_ratings.iloc[recommend[0][0]]['hotel id'].astype(int)))

Recommended hotel with hotel id 2514393


In [78]:
rec_hotel = hotel_ratings[hotel_ratings['hotel id'] == 2514393][['rooms', 'service', 'cleanliness', 'front desk', 'business service', 'value', 'location']]

We evaluate this recommendation by calculating the RMSE (the Root Mean Squared Error) to see how far off our recommended hotel's rating structure is from our favorite.

In [82]:
def rmse(rec, test):
    return np.sqrt(np.linalg.norm((rec - test))/7.0)

In [83]:
rmse(rec_hotel, test_hotel)

0.4860846722988549

<img src='radar.jpg' style="float: right; height: 400px;" alt="Drawing">
From over thousands of hotels, we have found a hotel (__Hotel Gansevoort__) that has been rate as highly on _value_, _cleanliness_, and _location_ almost same as our _favorite_ hotel, __Hotel Erwin, a Joie de Vivre hotel__. It took us only 10 microseconds wall time, instead of possibly hours of waiting for pairwise computation of similarities. Pairwise computing also eats up most of pc's memory and will cause memory error in python kernels.

__Next Steps__:
1. Use heapmax to get k nearest neighbors, say, 5 most similar hotels based on the 7 dimensions.
2. Optimize hyper-parameter k by running grid search and cross validation, evaluated by RMSE.
3. Can run this on subsets of hotels that are in the same zipcode.